Tips:

* Enable a GPU in Colab before running the neural network examples in this notebook. *Edit -> Notebook settings -> Hardware accelerator -> GPU.* 

* Should you need to reset your environment to a clean state, you can use *Runtime -> Disconnect and delete runtime*.

* You can find GPU info by running `!nvidia-smi`.

# LEAP, May 2022 🌎: Trees and Neural Networks with TensorFlow 

Welcome! Today, you'll gain hands-on experience training decision forests and neural networks with TensorFlow. Trees and neural networks are two of the most [frequently](https://www.kaggle.com/kaggle-survey-2021) used models on [Kaggle](https://kaggle.com/). Together, they provide an intro to practical ML. 

This notebook contains tutorials and exercises to help you get started. Your instructor will guide you through the sections you'll explore today. 

If you're new to deep learning (or ML in general), this is a *lot* of material to cover in a short workshop. Our goals are to dive in and get started. 

Here's an outline of what we'll cover:

1. **Decision Forests**: You'll train a random forest on a tabular dataset that you load from a CSV file. This is a common pattern in practice. As an exercise, you'll train a gradient boosted tree.

1. **Deep Neural Networks:** You'll train a DNN to classify handwritten digits. This is the "hello world" of computer vision, and a great place to begin if you're new to the subject. As an exercise, you'll experiment with a different dataset, and modify the network to improve the accuracy.

1. **Transfer Learning:** You will download a pretrained model, use it to classify images, then add a classification head for new dataset.

1. **Convolutional Neural Networks:** You'll write a CNN to classify images of cats and dogs, using a real-world dataset you load from a directory on disk. As an exercise, you'll use data augmentation and dropout to reduce overfitting.

1. **DeepDream:** Deep learning is an amazingly wide field. If time remains, your instructor will walk you through DeepDream, which is a different flavor than a classification or regression task. 

Okay, let's get started!

# Random Forests

## 🌲🌳🌲🌳🌲🐿️🐻

Decision Forests are a family of tree-based models including Random Forests and Gradient Boosted Trees. They are the most popular class of models on Kaggle, and are the best place to start when working with tabular data.

You will use TensorFlow to train each of these on a dataset you load from a CSV file. This is a common pattern in practice. Roughly, your code will look as follows:

```
import tensorflow_decision_forests as tfdf
import pandas as pd
  
dataset = pd.read_csv("project/dataset.csv")
tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(dataset, label="my_label")

model = tfdf.keras.RandomForestModel()
model.fit(tf_dataset)
  
print(model.summary())
```

## Install software

There are many excellent libraries for working with tree-based models, including [scikit-learn](https://scikit-learn.org/) (highly recommended for all your ML needs), XGBoost, LightGBM, and others.

Today, you'll use [TensorFlow Decision Forests (TF-DF)](https://www.tensorflow.org/decision_forests), a  library used in production at Google to train large models. The open-source release is currently in beta. 

If you use TF-DF in your climate research, we would *love* to hear about it. And/or, if you encounter bugs or friction not mentioned in the notes here, please email Josh.

Let's install TF-DF now.

In [ ]:
!pip install tensorflow_decision_forests --quiet --upgrade

## Import the library

You may see a warnings about certain distrubuted training modes not being available during the beta. That's expected, and you can safely ignore these.

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd

In [ ]:
print("TensorFlow v" + tf.__version__)
print("TensorFlow Decision Forests v" + tfdf.__version__)

## Download the penguins dataset

To start, you will work with a small tabular [dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) of about 300 penguins. You will predict the species of penguin (Adelie, Gentoo, or Chinstrap) based on numeric attributes like their flipper length, and categorical attributes like the name of the island they're found on.

In [ ]:
# Download the dataset
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

# Load a dataset into a Pandas Dataframe
dataset_df = pd.read_csv("/tmp/penguins.csv")

# Display the first 3 examples
dataset_df.head(3)

## Prepare the dataset

This dataset contains a mix of numeric (*bill_depth_mm*), categorical (*island*) and missing features. TF-DF supports all these feature types natively, and no preprocessing is required. This is one of the advantages of tree-based models, and why they're a great place to start.

You will have to slightly adjusted the labels, though, to convert them into the integer format TF-DF expects. The label (species) is stored as a string, so let's convert that into an integer.


In [ ]:
label = "species"

classes = dataset_df[label].unique().tolist()
print(f"Label classes: {classes}")

dataset_df[label] = dataset_df[label].map(classes.index)

Next, split the dataset into training and testing:

In [ ]:
import numpy as np

def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

There's one more step required before you can train your model. You need to convert from Pandas format (`pd.DataFrame`) into TensorFlow format (`tf.data.Dataset`). We've provided a single line helper function that will do this for you: 

```
tfdf.keras.pd_dataframe_to_tf_dataset(your_df, label='species')
```

This is a high [performance](https://www.tensorflow.org/guide/data_performance) data loading library which is helpful when training neural networks with accelerators like GPUs and TPUs. It it not necessary for tree-based models until you begin to do distributed training - but we'll use it today for practice.

Creating a fast input pipeline is important when working with neural networks, and forgetting to do so is the most common bug new researchers encounter. The author of this notebook has seen many folks with expensive GPUs that are idle ~50% of the time while waiting for data.

Note that tf.data is a bit tricky to use, and has a learning curve. There are guides on [tensorflow.org/guide](https://www.tensorflow.org/guide) to help.

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

## What models are available?

There are several tree-based models for you to choose from. To start, you'll work with a Random Forest. Thus is the most well-known of the Decision Forest training algorithms. 

A Random Forest is a collection of decision trees, each trained independently on a random subset of the training dataset (sampled with replacement). The algorithm is unique in that it is robust to overfitting, and easy to use.

In [ ]:
tfdf.keras.get_all_models()

Unlike neural networks, decision forests have relatively few (and easy to configure) hyperparameters with good defaults.

## How can I configure them?

TF-DF provides good defaults for you (e.g. the top ranking hyperparameters on our benchmarks, slightly modified to run in reasonable time). You will use these defaults below. If you would like to configure the learning algorithm, you will find many options you can explore to get the highest possible accuracy. 

Let's check out the help on the ```RandomForestModel``` to see the options.

In [ ]:
help(tfdf.keras.RandomForestModel)

There are **many** hyperparamters you can explore to grow exactly the type of forest you like. 


In [ ]:
# You can the parameters as follows
print(tfdf.keras.RandomForestModel.predefined_hyperparameters())

You can select a template and/or set parameters as follows:

```gbt = tfdf.keras.GradientBoostedTreesModel(hyperparameter_template="benchmark_rank1",num_trees=300)```


## Create a Random Forest 

Today, you will use the defaults. Let's create your model. 

In [ ]:
rf = tfdf.keras.RandomForestModel()
rf.compile(metrics=["accuracy"]) # Optional, you can use this to include a list of eval metrics

## Train your model

This is a one-liner.

Note: you may see a warning about Autograph. You can safely ignore this, it will be fixed in the next release.

In [ ]:
rf.fit(x=train_ds)

## Visualize your model
One benefit of tree-based models is that you can easily visualize them. The default number of trees used in the Random Forest is 300. You can select a tree to display below.

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)

## Evaluate the model on OOB data and the test dataset

Let's plot accuracy on OOB evaluation dataset as a function of the number of trees in the forest. One of the nice features about this particular hyperparameter is that larger values are usually better, and come with little risk aside from slowing down training.


In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")
plt.show()

You can also see some general stats on the OOB dataset:

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()

Now, let's run an evaluation using the test data. Depending on the random split your accuracy will likely between 90-100%.

In [ ]:
evaluation = rf.evaluate(x=test_ds,return_dict=True)

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

## Variable importances

There are several ways to identify important features. Let's see the available options:

In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

Let's display one of them:

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

## Predict on a single example

Here's example code you can use to make predictions on a single example. Note that TensorFlow is optimized for batch prediction. This code below is mainly helpful for experimenting.

In [ ]:
# Create your example as a dictionary
example = {"bill_depth_mm" : [0],
           "bill_length_mm" : [0],
           "body_mass_g" : [0],
           "flipper_length_mm" : [0],
           "island" : ["Torgersen"],
           "sex" : "female",
           "year" : 2007}

# Convert the dictionary into a DataFrame
example_df = pd.DataFrame.from_dict(example)

# Convert the DataFrame into tf.data format
example_ds = tfdf.keras.pd_dataframe_to_tf_dataset(example_df)

# Call predict
rf.predict(example_ds)

## Predict on many examples

Following is code you can use to display predictions for each example in the test set. Note that similar code will be a bit different for neural networks, which typically use a different data structure inside tf.data to pack the features and labels.

In [ ]:
# Make predictions on every example in the test set
predictions = rf.predict(test_ds)

# Loop over the test set, and display the predicted value and label
features, labels = next(iter(test_ds))
for pred, label in zip(predictions, labels):
  print ("Pred:", np.argmax(pred), "Actual:", label.numpy())

# Exercise: Gradient Boosted Trees

In this exercise you will download the [census](https://archive.ics.uci.edu/ml/datasets/census+income) dataset which. This contains ~40K examples with a mix of numeric and categorical attributes. You will train a gradient boosted tree, identify important features, and evaluate your model's accuracy.

We've provided a bunch of code you can use to explore the dataset, in case this is helpful to you in your future work. The code you need to write for this exercise is only a couple lines.

Notes:
- You can visualize this dataset in your browser using https://pair-code.github.io/facets/ 
- This dataset has fairness concerns, which you can learn about at https://www.tensorflow.org/responsible_ai

### Instructions

Complete the code cells below. See the comments for instructions. You can find a solution at the end.

### Download and explore the dataset

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/census/adult.data -O /tmp/adult.csv

In [ ]:
# Take a look at the CSV
!head /tmp/adult.csv

In [ ]:
# The CSV is missing a header
cols = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label'
]

df = pd.read_csv("/tmp/adult.csv", names=cols)

In [ ]:
# Clean up

# Drop a meaningless column
df = df.drop(columns=['fnlwgt'])

# Convert the label into integer format
df["label"] = df["label"].apply(lambda x: 1 if x == ' >50K' else 0).values

# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
label_col = 'label'
categorical_columns = list(df.select_dtypes(include='object').columns)
numeric_columns = [c for c in df.columns if c not in categorical_columns]

print('Categorical columns', categorical_columns)
print('Numeric columns', numeric_columns)

feature_columns = categorical_columns + numeric_columns

In [ ]:
train_df, test_df = split_dataset(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_df), len(test_df)))

In [ ]:
train_df[numeric_columns].describe()

In [ ]:
train_df[categorical_columns].nunique()

In [ ]:
for col in categorical_columns:
  print(col, list(train_df[col].unique()))

What is the class balance?

In [ ]:
train_df["label"].sum() / len(train_df)

In [ ]:
test_df["label"].sum() / len(test_df)

In [ ]:
print('Train shape:', train_df.shape)
print('Test shape :', test_df.shape)

Create tf.data.Datasets from the Pandas DataFrame, using the one-liner shown above.

In [ ]:
# YOUR CODE HERE
# Add code to create a tf.data.Dataset for train and test from the DataFrames

# Example...
# train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(...
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(...

### Create and train your model

Create your model. You can write a one-liner for this, similar to the example above. Previously, you learned how to work with Random Forests. If you like, you can create a Random Forest again, or you can try creating a Gradient Boosted Tree. 

As a reminder, you can see which models are available by running `tfdf.keras.get_all_models()`, or visiting [tensorflow.org/decision_forests](https://www.tensorflow.org/decision_forests).

In [ ]:
# YOUR CODE HERE
# Add code to create a gradient boosted tree
# Example ...
# gbt = tfdf.keras. ...
# gbt.compile(metrics=["accuracy"])

Train your model. You can write a one-liner for this, similar to the example above.

In [ ]:
# YOUR CODE HERE
# Add code to train your model
# Example ...
# gbt.fit(...

### Evaluate your model

Uncomment these cells after completing the code above.

In [ ]:
#gbt.summary()

In [ ]:
#gbt.evaluate(test_ds)

In [ ]:
# evaluation = gbt.evaluate(x=test_ds,return_dict=True)

# for name, value in evaluation.items():
#   print(f"{name}: {value:.4f}")

In [ ]:
# inspector = gbt.make_inspector()
# inspector.evaluation()

In [ ]:
# inspector.variable_importances()["NUM_AS_ROOT"]

In [ ]:
# tfdf.model_plotter.plot_model_in_colab(gbt, tree_idx=0, max_depth=3)

### Solution


**Create tf.data.Datasets from the pd.DataFrame**

To do so, you can write:

```
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="label")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="label")
```

If you will be working with the same tf.data.Dataset multiple times, you can add `.cache()` at the end of those lines to keep it in memory.

**Create and train a GradientBoostedTrees model**

To do so, you can write:

```
gbt = tfdf.keras.GradientBoostedTreesModel()
gbt.compile(metrics=["accuracy"])
gbt.fit(train_ds)
```


### Next steps

**Try a larger dataset**

Thanks to our friends at Kaggle, you can find a tabular dataset with ~1.7M rows and starter code for TensorFlow Decision Forests [here](https://www.kaggle.com/code/paultimothymooney/getting-started-with-tensorflow-decision-forests/). This is great if you'd like to start running larger experiments.

**Hyperparameter tuning**

You can use [Keras Tuner](https://keras.io/keras_tuner/) for easy hyperparameter optmization. 

# MNIST


## ✍️0️⃣1️⃣2️⃣3️⃣4️⃣5️⃣6️⃣8️⃣9️⃣🔟

Training an image classifier on the MNIST dataset of handwritten digits is considered the "hello world" of computer vision. In this tutorial, you will download the dataset, then train a linear model, a neural network, and a deep neural network to classify it. 

**Key point:** Deep Learning is "code light, but concept heavy". You'll be able to implement a Deep Neural Network in about five lines of code, but the underlying concepts (cross-entropy, softmax, dense layers, etc) normally take a few months to learn. You do need to understand these all today to dive in.

## Import TensorFlow

In [ ]:
import tensorflow as tf
print("You are using TensorFlow version", tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
  print("You have a GPU enabled.")
else:
  print("Enable a GPU before running this notebook.")

Colab has a variety of GPU types available (each new  instance is assigned one randomly, depending on availability). To see which type of GPU you have, you can run ```!nvidia-smi``` in a code cell. Some are quite fast!

In [ ]:
# In this notebook, we'll use Keras: TensorFlow's user-friendly API to 
# define neural networks. Let's import Keras now.
from tensorflow import keras
import matplotlib.pyplot as plt

## Download the MNIST dataset
MNIST contains 70,000 grayscale images in 10 categories. The images are low resolution (28 by 28 pixels). An important skill in Deep Learning is exploring your dataset, and understanding the format. Let's download MNIST, and explore it now.

In [ ]:
dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()

There are 60,000 images in the training set:

In [ ]:
print(train_images.shape)

And 10,000 in the testing set:

In [ ]:
print(test_images.shape)

Each label is an integer between 0-9:

In [ ]:
print(train_labels)

## Preprocess the data
The pixel values in the images range between 0 and 255. Let's normalize the values 0 and 1 by dividing all the images by 255. It's important that the training set and the testing set are preprocessed in the same way.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Let's display the first 25 images from the training set, and display the label below each image.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

## Create the layers

Neural networks are made up of layers. Here, you'll define the layers, and assemble them into a model. We will start with a single Dense layer. 

### What does a layer do?

The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. For example:

- The first layer in a network might receives the pixel values as input. From these, it learns to detect edges (combinations of pixels). 

- The next layer in the network receives edges as input, and may learn to detect lines (combinations of edges). 

- If you added another layer, it might learn to detect shapes (combinations of edges).

The "Deep" in "Deep Learning" refers to the depth of the network. Deeper networks can learn increasingly abstract patterns. Roughly, the width of a layer (in terms of the number of neurons) refers to the number of patterns it can learn of each type.

Most of deep learning consists of chaining together simple layers. Most layers, such as [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), have parameters that are initialized randomly, then tuned (or learned) during training by gradient descent.

In [ ]:
# A linear model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(10, activation='softmax')
])

The first layer in this network, [tf.keras.layers.Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data. This is necessary since Dense layers require arrays as input.

After the pixels are flattened, this model consists of a single Dense layer. This is a densely connected, or fully connected, neural layer. The Dense layer has 10 neurons with softmax activation. This returns an array of 10 probability scores that sum to 1. 

After classifying an image, each neuron will contains a score that indicates the probability that the current image belongs to one of the 10 classes.

## Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

*Loss function* — This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.

*Optimizer* — This is how the model is updated based on the data it sees and its loss function.

*Metrics* — Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the model
Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the ```train_images``` and ```train_labels``` arrays.

1. The model learns to associate images and labels.

1. You ask the model to make predictions about a test set—in this example, the ```test_images``` array.

1. Verify that the predictions match the labels from the ```test_labels``` array.

To begin training, call the ```model.fit``` method — so called because it "fits" the model to the training data:

In [ ]:
EPOCHS=10
model.fit(train_images, train_labels, epochs=EPOCHS)

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.90 (or 90%) on the training data. Accuracy may be slightly different each time you run this code, since the parameters inside the Dense layer are randomly initialized.

## Evaluate accuracy
Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy:', test_acc)

It turns out that the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents overfitting. Overfitting is when a machine learning model performs worse on new, previously unseen inputs than on the training data. An overfitted model "memorizes" the training data—with less accuracy on testing data. 

## Make predictions
With the model trained, you can use it to make predictions about some images.

In [ ]:
predictions = model.predict(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [ ]:
print(predictions[0])

A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 10 digits. You can see which label has the highest confidence value:

In [ ]:
print(tf.argmax(predictions[0]))

# Exercise: Fashion MNIST

## 👕👟👔👗

In the above tutorial, you trained a linear model (a single Dense layer) on the MNIST dataset. As an exercise, let's modify your code above to:
- Use a new dataset (Fashion MNIST)
- Train a neural network (with two Dense layers, instead of just one)
- Create plots to observe overfitting and underfitting

## Instructions

You will need to make two changes in the code above.

**1) Import the Fashion MNIST** 

To do so, change the line

```
dataset = keras.datasets.mnist
``` 

to 

```
dataset = keras.datasets.fashion_mnist
```

**2) Modify the model definition to create a neural network**

To do so, change the lines from:

```
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(10, activation='softmax')
])
```

to

```
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
```

This will define a neural network with a single hidden layer. If you like, you can experiment by adding a third Dense layer, which will create a deep neural network. For example:

```
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
```

After making the above changes, on the Colab menu   select *Edit -> Clear all outputs* and *Runtime -> Restart runtime* to restore this notebook to a clean state. Run the cells in the tutorial above to train your neural network on Fashion MNIST.

**3) Add plots to observe overfitting**

If trained for too long, a NN may begin to memorize the training data (rather than learning patterns that generalize to unseen data). This is called overfitting. Of all the hyperparmeters in the design of your network (the number and width of layers, the optimizer, etc) - the most important to set properly is ```epochs```. You will learn more about this in exercise two.

To create plots to observe overfitting, modify your training loop as follows.

Change:

```
history = model.fit(train_images, train_labels, epochs=EPOCHS)
```

to:

```
history = model.fit(train_images, train_labels, 
                    validation_data=(test_images, test_labels),
                    epochs=EPOCHS)
```

This will capture the accuracy and loss on the training and validation data after epoch. To plot the results, create a new code cell, and add the following code:

```
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
```



# Game break: Teachable Machine
If you'd like, now would be a good time to take a break from coding and try: https://teachablemachine.withgoogle.com/

# Transfer Learning

You don't always need to write models totally from scratch.

With [transfer learning](https://developers.google.com/machine-learning/glossary#transfer-learning), you can take knowledge that a neural network has learned from one task and apply it to a different (but related) task.

The way we do this is by using a _pre-trained model_. With transfer learning, you leverage the trained layers of the pre-trained model as a feature extraction, and add additional layers to fit your application.

In this example, you'll use a model that has been trained on the ImageNet dataset. [ImageNet](https://www.image-net.org/) is a common image classification benchmark dataset that contains ~1M images of animals, furniture, clothing, and more.

You'll use this pre-trained model to classify images from the TensorFlow Flowers dataset, which contains thousands of images of different flower types. You'll then fine-tune the pre-trained model for higher accuracy.

## 🌹 🌸 🌺 🌼 🌻

We'll download the pre-trained model from [TensorFlow Hub](https://tfhub.dev/), which is a repository of pre-trained TensorFlow models.

You'll learn how to:

1. Use models from TensorFlow Hub with `tf.keras`.
1. Use an image classification model from TensorFlow Hub.
1. Do  transfer learning to fine-tune a model for your own image classes.

In [ ]:
import numpy as np

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

## Get Predictions from a pre-trained model

Some of the models in TensorFlow Hub contain the classification head. These models can be used without any retraining. The caveat is that they will only be able to make predictions for labels they have seen in the training set. So if your pretrained model was trained only on dog images, but you want to get predictions for flowers, it's not going to work well.

Let's first define the path to the model in TensorFlow Hub that we want to use.

Feel free to click the link below and read more about the model on the TF Hub site.

We'll use [MobileNet V2](https://arxiv.org/abs/1704.04861), which is a computer vision model for mobile or embedded applications (think: running on your phone).

In [ ]:
hub_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

Next, we wrap this pre-trained model from TensorFlow Hub as a Keras layer with `hub.KerasLayer` and create a model using the `tf.keras.Sequential` API.

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(hub_model, input_shape=IMAGE_SHAPE+(3,))
])

Let's download an image of Grace Hopper and get a prediction from this pre-trained MobileNet model.

In [ ]:
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

We need to scale the image by dividing by 255.
Then we call the predict method on our model, passing in the image.

In [ ]:
grace_hopper = np.array(grace_hopper) / 255.0
result = classifier.predict(grace_hopper[np.newaxis, ...])

The result is a 1001-element vector, rating the probability of each class for the image.

The top class ID can be found with tf.math.argmax:

In [ ]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class.numpy()

Take the predicted_class ID (such as 653) and fetch the ImageNet dataset labels to decode the predictions:

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())


## Exercise: Try a different pre-trained model

In the above tutorial, you used a pre-trained MobileNet model to get predictions for an imge. TensorFlow Hub contains *many* image models you can use.

In this exercise, you'll modify the code to:
- Use a different pre-trained model called [Inception V3](https://arxiv.org/abs/1409.4842)


## Instructions

You'll need to make one change in the code above.

**1) Specify a different model URL** 

To do so, change the line

```
hub_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
``` 

to 

```
hub_model = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"
```

You'll notice that even though the Inception model and the MobileNet model were both trained on the ImageNet dataset, they will produce different predictions for the Grace Hopper image.

There are **many** models on TensorFlow Hub you can use for your image classification needs, including many state-of-the-art models. To use them, you generally need to change the model URL (as above), and use different image preprocessing (to convert your data into the format the model expects).

Now let's continue and learn how to customize one of these models to work better for your domain. 

## Add a classification head

Using the pre-trained model was fast, but not very helpful if we have labels that weren't included in the training set.

However, we can still make use of the knowledge that the pre-trained model has learned from looking at the 1M+ images in the ImageNet dataset. To do this, we will add a classification layer to the pre-trained model, and then retrain just that final layer.

### Download the dataset

First, load this data into the model using the image data off disk with `tf.keras.utils.image_dataset_from_directory`.

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

The flowers dataset has five classes:

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

Let's scale our data by 255 using `tf.keras.layers.Rescaling`

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

Let's check out the shape of the images and the labels

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

Before we train the model, let's quickly see what the predictions look like on this Flowers dataset from the classifier we used in the previous section.

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]
predicted_class_names

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

The results are far from perfect, but reasonable considering that these are not the classes the model was trained for (except for "daisy").

To improve our results, it's time to train on our data!

### Download the headless model

As before, we need to download the classifier. Notice that this time, the path does not contain _"classification"_. Instead it is a _"feature vector"_. 

TensorFlow Hub provides feature vectors, which are pre-trained models without the top classification layer. This is what we want if we plan to do any retraining.

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

As before, wrap the pre-trained model with `hub.KerasLayer`.

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

The feature extractor returns a 1280-element vector for each image (the image batch size remains at 32 in this example):

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

Next, add a classification layer and create a model with the Keras Sequential API.

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.summary()

From `model.summary` you can see that the architecture of the model is the TF Hub model followed by a dense layer with 5 units (because we have 5 different types of flowers in our dataset).

Next, we compile and fit our model. Make sure you're using a GPU or this could take a while...

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [ ]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS)

### Check the predictions

Obtain the ordered list of class names from the model predictions:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

Plot the model predictions:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

These labels look much more reasonable!

### Next steps

If you have a large dataset (say, of ~1M images) you can retrain the entire model you downloaded from TensorFlow Hub from scratch. This may give higher accuracy than using the pre-trained weights, but will take a while.

```
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=True)
```

## Learning more

There are many resources on https://www.tensorflow.org/hub you can use to learn more. To start, you may like to check out the image classification [tutorial](https://www.tensorflow.org/hub/tutorials/image_classification) on TensorFlow Hub. If you're interested in NLP, you can check out the tutorials for [BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

# Cats and Dogs

## 🐈🐕🐱🐶

 ## Download and explore the dataset

Although you are downloading large files, you are doing so in Colab through Google Cloud Platform (instead of over your local WiFi connection). This means that downloads will usually be fast, regardless of your internet connection.

In [ ]:
import os

In [ ]:
# Our dataset is a zip on the web
origin = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=origin, extract=True)
path_to_folder = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

The unzipped dataset has the following directory structure:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

The dataset is divided into train and validation splits. Let's create variables that point to each of these directories.

In [ ]:
train_dir = os.path.join(path_to_folder, 'train')
validation_dir = os.path.join(path_to_folder, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Now let's count the number of images in each directory.

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print('Total training cat images:', num_cats_tr)
print('Total training dog images:', num_dogs_tr)
print('Total validation cat images:', num_cats_val)
print('Total validation dog images:', num_dogs_val)
print('---')
print("Total training images:", total_train)
print("Total validation images:", total_val)

You should see that we have 3,000 total images (2,000 in train and 1,000 in validation). Note that this dataset is balanced (we have an equal number of cats and dogs).

Tip: in addition to Python, you can run shell commands in Colab (for example, ```!ls $train_cats_dir```).

In [ ]:
!ls $train_cats_dir 

Let's display a couple images.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
_ = plt.imshow(plt.imread(os.path.join(train_cats_dir, "cat.0.jpg")))

In [ ]:
_ = plt.imshow(plt.imread(os.path.join(train_cats_dir, "cat.1.jpg")))

Note that the images are different sizes. Before feeding them into a CNN, we'll need to reshape them all to the same dimensions. We'll take care of that in the next section.

## Data preprocessing

Next, we will need a way to read these images off disk, and to preprocess them. Specifically, we will need to:
- Read the image off disk.
- Decode contents of these images and convert them into RGB arrays.
- Convert the pixels values from integer to floating point types.
- Rescale the pixel from values between 0 and 255 to values between 0 and 1 (neural networks work better with small input values - under the hood, each input is multiplied by a weight, large inputs could result in overflow).

Fortunately, all of these tasks can be done with the `ImageDataGenerator` class provided by `tf.keras`. It can read images from disk and preprocess them into proper arrays.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Let's resize images to this size
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
# Rescale the pixel values to range between 0 and 1
train_generator = ImageDataGenerator(rescale=1./255)
val_generator = ImageDataGenerator(rescale=1./255)

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [ ]:
batch_size = 32 # Read a batch of 64 images at each step

In [ ]:
train_data_gen = train_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

In [ ]:
val_data_gen = val_generator.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Use the generators to display a few images and their labels

Next, we will extract a batch of images from the training generator, then plot several of them with `matplotlib`. The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is the pixel values and y_train is the labels.

In [ ]:
image_batch, labels_batch = next(train_data_gen)

In [ ]:
# The shape will be (32, 150, 150, 3)
# This means a list of 32 images, each of which is 150x150x3.
# The 3 at the end refers to the R,G,B color channels.
# A grayscale image would be (for example) 150x150x1
print(image_batch.shape)

In [ ]:
# The shape (32,) means a list of 64 numbers
# each of these will either be 0 or 1
print(labels_batch.shape)

In [ ]:
# This function will plot images returned by the generator
# in a grid with 1 row and 5 columns
def plot_images(images):
  fig, axes = plt.subplots(1, 5, figsize=(10,10))
  axes = axes.flatten()
  for img, ax in zip(images, axes):
      ax.imshow(img)
      ax.axis('off')
  plt.tight_layout()
  plt.show() 

In [ ]:
plot_images(image_batch[:5])

Next, let's retrieve the labels. All images will be labeled either 0 or 1, since this is a binary classification problem. 

In [ ]:
# Here are the first 5 labels from the dataset
# that correspond to the images above
print(labels_batch[:5])

In [ ]:
# Here, we can see that "0" maps to cat,
# and "1" maps to dog
print(train_data_gen.class_indices)

## Create the model
Your model will consist of three convolutional blocks followed by max pooling. There's a fully connected layer with 256 units on top. This model will output class probabilities (between 0 and 1) based on the `sigmoid` activation function. If the output is closer to 1, the image will be classified as a dog, otherwise a cat. 

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

Compile the model, and select the adam optimizer for gradient descent, and binary cross entropy for the loss function (roughly, cross entropy is a way to measure the distance between the prediction we wanted the network to make, and the prediction it made).

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Let's look at a diagram of all the layers of the network using the `summary` method:

In [ ]:
model.summary()

This model has about 5M parameters (or weights) to learn. Our model is ready to go, and next we can train it using the data generators we created earlier.

## Train the model

Use the `fit` method to train the network. You will train the model for 15 epochs (an epoch is one "sweep" over the training set, where each image is used once to perform a round of gradient descent, and update the models parameters). This will take one to two minutes, so let's start it now:

In [ ]:
epochs = 15

In [ ]:
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
)

Inside `model.fit`, TensorFlow uses gradient descent to find useful values for all the weights in the model. When you create the model, the weights are initialized randomly, then gradually improved over time. The data generator is used to load batches of data off disk. Then, for each batch:
- The model performs a forward pass (the images are classified by the network).
- Then, the model performs a backward pass (the error is computed, then each weight is slightly adjusted using gradient descent to improve the accuracy on the next iteration).

Gradient descent is an iterative process. The longer you train the model, the more accurate it will become on the training set. But, the more likely it is to overfit! Meaning, the model will begin to memorize the training images, rather than learn patterns that enable it generalize to new images not included in the training set. 

- We can see whether overfitting is present by comparing the accuracy on the training and validation data.

If you look at the accuracy figures reported above, you should see that training accuracy is over 90%, while validation accuracy is only around 70%.

## Create plots to check for overfitting
Accuracy on the validation data is important: it helps you estimate how well our model is likely to work on new, unseen data in the future. To see how much overfitting is present (and when it occurs), we will create two plots, one for accuracy, and another for loss. Roughly, loss (or error) is the inverse of accuracy (lower is better). Unlike accuracy, loss takes the confidence of a prediction into account (a confidently wrong predicitions has a higher loss than one that is only slightly wrong).

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Overfitting occurs when the validation loss stops decreasing. In this case, that occurs around epoch 5 (give or take). Your results may be slightly different each time you run this code (since the weights are initialized randomly).

Why does overfitting happen? When there are only a "small" number of training examples, the model sometimes learns from noises or unwanted details, to an extent that it negatively impacts the performance of the model on new examples. It means that the model will have a difficult time "generalizing" on a new dataset (making accurate predictions on images that weren't included in the training set).

# Game break: Quick, Draw!
If you'd like, now would be a good time to take a break from coding and try: https://quickdraw.withgoogle.com/

# Exercise: Reduce overfitting

## Instructions

In this exercise, you will use data augmentation and dropout to improve your model. Follow along by reading and running the code below. There are two **TODOs** for you to complete, and a solution is given below.

## Data augmentation
Overfitting occurs when there are a "small" number of training examples. One way to fix this problem is to increase the size of the training set, by gathering more data (the larger and more diverse the dataset, the better!)

We can also use a technique called "data augmentation" to increase the size of the training set, by generating new examples from existing ones by applying random transformations (for example, rotation) that yield believable-looking images. 

This is especially effective when working with images. For example, our training set may only contain images of cats that are right side up. If our validation set contains images of cats that are upside down, our model may have trouble classifying them correctly. To help teach it that cats can appear in any orientation, we will randomly rotate images from our training set during training. This helps expose the model to more aspects of the data, and can lead to better generalization.

Data augmentation is built into the ImageDataGenerator. You can specifiy different transformations, and it will take care of applying then during the training.

In [ ]:
# Let's create new data generators, this time with 
# data augmentation enabled
train_generator = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [ ]:
train_data_gen = train_generator.flow_from_directory(batch_size=32,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

The next cell will show how the same training image appears when used with five different types of data augmentation.

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plot_images(augmented_images)

We only apply data augmentation to the training examples, so our validation generator looks the same as before.

In [ ]:
val_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
val_data_gen = val_generator.flow_from_directory(batch_size=32,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Dropout

Another technique to reduce overfitting is to introduce dropout to the network. Dropout is a form of regularization that makes it more difficult for the network to memorize rare details (instead, it is forced to learn more general patterns).

When you apply dropout to a layer it randomly drops out (set to zero) a number of activations during training. Dropout takes a fractional number as its input value, in the form such as 0.1, 0.2, 0.4, etc. This means dropping out 10%, 20% or 40% of the output units randomly from the applied layer.

When appling 0.1 dropout to a certain layer, it randomly deactivates 10% of the output units in each training epoch.

Create a new model using Dropout. You'll reuse the model definition from above, and add a Dropout layer.

In [ ]:
from tensorflow.keras.layers import Dropout

In [ ]:
# TODO: Your code here
# Create a new CNN that takes advantage of Dropout.
# 1) Reuse the model declared in tutorial above.
# 2) Add a new line that says "Dropout(0.2)," immediately
# before the line that says "Flatten()".

## Solution


In [ ]:
#@title
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

After introducing dropout to the network, compile your model and view the layers summary. You should see a Dropout layer right before flatten.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

## Train your new model
Add code to train your new model. Previously, we trained for 15 epochs. You will need to train this new modek for more epochs, as data augmentation and dropout make it more difficult for a CNN to memorize the training data (this is what we want!).

Here, you'll train this model for 25 epochs. This may take a few minutes, and you may need to train it for longer to reach peak accuracy. If you like, you can continue experimenting with that at home.

In [ ]:
epochs = 25

In [ ]:
# TODO: your code here
# Add code to call model.fit, using your new
# data generators with image augmentation
# For reference, see the "Train the model"
# section above

## Solution

In [ ]:
#@title
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
)

## Evaluate your new model
Finally, let's again create plots of accuracy and loss (we use these plots often in practice!) Now, compare the loss and accuracy curves for the training and validation data. Were you able to achieve a higher validation accuracy than before? Note that even this model will eventually overfit. To prevent that, we use a technique called early stopping (we stop training when the validation loss is no longer decreasing). 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Game break: Sketch-RNN
If you'd like, now would be a good time to take a break from coding and try: https://magenta.tensorflow.org/assets/sketch_rnn_demo/index.html

# DeepDream

## 🛌💭

If time remains, in this tutorial your instructor will walk you through a minimal version of DeepDream, an experiment to visualize some of the features a convolutional neural network has learned to detect. DeepDream is an advanced tutorial, and our goal is to introduce you to some of the fascinating (and unexpected) things you can explore with Deep Learning. 

Normally, when training a model we use gradient descent to minimize classification loss. In a CNN, this means we adjust the weights in the filters. In DeepDream, we start with a large, pretrained CNN (and leave the filters fixed!) We then use gradient descent to modify the input image to increasingly activate the filters. For example, if there is a filter that recognizes a certain kind of texture, we can progressively modify the image to contain more and more examples of that texture.

In [ ]:
import numpy as np
from IPython.display import clear_output

## Download and display an image

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'

In [ ]:
def download(url, target_size=None):
  name = url.split('/')[-1]
  image_path = tf.keras.utils.get_file(name, origin=url)
  return tf.keras.preprocessing.image.load_img(image_path, target_size)

def show(img):
  plt.figure(figsize=(8,8))
  plt.grid(False)
  plt.axis('off')
  plt.imshow(img)
  plt.show()

original_img = download(url, target_size=[225, 375])
original_img = np.array(original_img)
show(original_img)

## Rescale the pixel values

In [ ]:
def preprocess(img):
  """ Convert RGB values from [0, 255] to [-1, 1] """
  img = tf.cast(img, tf.float32)
  img /= 128.0
  img -= 1.
  return img

def unprocess(img):
  """ Undo the preprocessing above """
  img = 255 * (img + 1.0) / 2.0
  return tf.cast(img, tf.uint8)

## Import a large, pretrained CNN
This model has been trained on ImageNet, a dataset with about 1M images in about 1K classes

In [ ]:
conv_base = tf.keras.applications.InceptionV3(weights='imagenet', 
                                              include_top=False)

## Choose layers to activate
Normally, when you train a neural network, you use gradient descent to adjust the weights to minimize loss, in order to accurately classify images. In DeepDream, the trick is to use gradient descent to adjust the **image**, in order to increasingly activate certain layers from the network. You can explore different layers and see how this affects the results. You can find all the layer names using ```model.summary()```. 

In [ ]:
names = ['mixed2', 'mixed3', 'mixed4', 'mixed5']
layers = [conv_base.get_layer(name).output for name in names]
model = tf.keras.Model(inputs=conv_base.input, outputs=layers)

## Custom loss function
Normally, we would use cross-entropy loss (for classification), or mean squared error (for regression). Here, we'll write a loss function that describes how activated our layers were by the image.

In [ ]:
def calc_loss(img):
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  losses = [tf.math.reduce_mean(act) for act in layer_activations]
  return tf.reduce_sum(losses)

## Use gradient ascent to progressively activate the layers
Normally, when training a model you use gradient *descent* to adjust the weights to reduce the loss. In DeepDream, you will use gradient *ascent* to maximize the activation of the layers you selected by modifying the image, while leaving the weights of the network fixed.

In [ ]:
@tf.function
def step(img, lr=0.001):
  with tf.GradientTape() as tape:
    loss = calc_loss(img)

  gradients = tape.gradient(loss, img)
  gradients /= tf.math.reduce_std(gradients) + 1e-8 

  # Because the gradients are in the same shape 
  # as the image, we can directly add them to it!
  img.assign_add(gradients * lr)
  img.assign(tf.clip_by_value(img, -1, 1))

In [ ]:
img = tf.Variable(preprocess(original_img))

steps = 1000
for i in range(steps):
  step(img)
  if i % 200 == 0:
    clear_output(wait=True)
    print ("Step {}".format(i))
    show(unprocess(img.numpy()))

clear_output(wait=True)
show(unprocess(img.numpy()))

You can find a complete example on the [website](https://www.tensorflow.org/tutorials/generative/deepdream) (which includes additional code to generate less noisy images), and you may also be interested in exploring a related technique [Neural Style Transfer](https://www.tensorflow.org/tutorials/generative/style_transfer).

# Learning more

📚

Book recommendations (both of these are **excellent**):

* [Deep Learning with Python, 2nd edition](https://www.manning.com/books/deep-learning-with-python-second-edition)
* [Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)


# Thank you!